# Wk4 : The Battle of Neighborhood

#### Importing libraries

In [1]:
import numpy as np 
import pandas as pd

import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

# Results from the previous assignment and clean-up

## 1. Clean up of the data

#### Making a dataframe from the raw data of Postal Code data from Toronto's Wikipedia

In [25]:
df_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

#### Dropping the rows with "Not assigned" in Borough

In [26]:
df_wiki.drop(df_wiki[df_wiki['Borough']=='Not assigned'].index, axis=0, inplace=True)
df_wiki = df_wiki.reset_index(drop=True)
#df_wiki

#### Number of the rows in the dataframe after the clean-up

In [27]:
df_wiki.shape

(103, 3)

## 2. Acquisition of geographical coordinates for each Postal Code

#### 1. Read the csv file of geographical coordinates of each postal code

In [28]:
df_Geocsv = pd.read_csv('Geospatial_Coordinates.csv')
df_Geocsv = df_Geocsv.reset_index(drop=True)
#df_Geocsv

#### 2. Merging wiki data and geographical coordinates data

#### Preparation of dataframe template

In [29]:
df = df_wiki
df['Latitude']=""
df['Longitude']=""
df.shape

(103, 5)

In [30]:
for i in range(0,df.shape[0]):
    for k in range(0,df_Geocsv.shape[0]):
        if (df_Geocsv['Postal Code'][k] == df['Postal Code'][i]):
            df['Latitude'][i] = df_Geocsv['Latitude'][k]
            df['Longitude'][i] = df_Geocsv['Longitude'][k]
            k =+ 1

#### The dataframe with Borough, Neighborhood and geographical coordinates

In [34]:
df.drop(['Postal Code'], axis=1,inplace=True)
df

Borough                                       Neighborhood  \
0          North York                                          Parkwoods   
1          North York                                   Victoria Village   
2    Downtown Toronto                          Regent Park, Harbourfront   
3          North York                   Lawrence Manor, Lawrence Heights   
4    Downtown Toronto        Queen's Park, Ontario Provincial Government   
..                ...                                                ...   
98          Etobicoke      The Kingsway, Montgomery Road, Old Mill North   
99   Downtown Toronto                               Church and Wellesley   
100      East Toronto  Business reply mail Processing Centre, South C...   
101         Etobicoke  Old Mill South, King's Mill Park, Sunnylea, Hu...   
102         Etobicoke  Mimico NW, The Queensway West, South of Bloor,...   

    Latitude Longitude  
0    43.7533  -79.3297  
1    43.7259  -79.3156  
2    43.6543  -79.3606  
3    43.7185  -79.4648  
4    43.6623  -79.3895  
..       ...       ...  
98   43.6537  -79.5069  
99   43.6659  -79.3832  
100  43.6627  -79.3216  
101  43.6363  -79.4985  
102  43.6288   -79.521  

[103 rows x 4 columns]

## 3. Exploring around Toronto

#### 1. Making the dataframe that contains "Toronto" in "Borough" cells

In [35]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [42]:
df_trnt=df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [43]:
df_trnt

Borough                                       Neighborhood  \
0   Downtown Toronto                          Regent Park, Harbourfront   
1   Downtown Toronto        Queen's Park, Ontario Provincial Government   
2   Downtown Toronto                           Garden District, Ryerson   
3   Downtown Toronto                                     St. James Town   
4       East Toronto                                        The Beaches   
5   Downtown Toronto                                        Berczy Park   
6   Downtown Toronto                                 Central Bay Street   
7   Downtown Toronto                                           Christie   
8   Downtown Toronto                           Richmond, Adelaide, King   
9       West Toronto                       Dufferin, Dovercourt Village   
10  Downtown Toronto  Harbourfront East, Union Station, Toronto Islands   
11      West Toronto                           Little Portugal, Trinity   
12      East Toronto                       The Danforth West, Riverdale   
13  Downtown Toronto           Toronto Dominion Centre, Design Exchange   
14      West Toronto       Brockton, Parkdale Village, Exhibition Place   
15      East Toronto                     India Bazaar, The Beaches West   
16  Downtown Toronto                     Commerce Court, Victoria Hotel   
17      East Toronto                                    Studio District   
18   Central Toronto                                      Lawrence Park   
19   Central Toronto                                           Roselawn   
20   Central Toronto                                   Davisville North   
21   Central Toronto    Forest Hill North & West, Forest Hill Road Park   
22      West Toronto                      High Park, The Junction South   
23   Central Toronto                  North Toronto West, Lawrence Park   
24   Central Toronto                The Annex, North Midtown, Yorkville   
25      West Toronto                             Parkdale, Roncesvalles   
26   Central Toronto                                         Davisville   
27  Downtown Toronto                     University of Toronto, Harbord   
28      West Toronto                                 Runnymede, Swansea   
29   Central Toronto                        Moore Park, Summerhill East   
30  Downtown Toronto          Kensington Market, Chinatown, Grange Park   
31   Central Toronto  Summerhill West, Rathnelly, South Hill, Forest...   
32  Downtown Toronto  CN Tower, King and Spadina, Railway Lands, Har...   
33  Downtown Toronto                                           Rosedale   
34  Downtown Toronto                                     Stn A PO Boxes   
35  Downtown Toronto                        St. James Town, Cabbagetown   
36  Downtown Toronto             First Canadian Place, Underground city   
37  Downtown Toronto                               Church and Wellesley   
38      East Toronto  Business reply mail Processing Centre, South C...   

   Latitude Longitude  
0   43.6543  -79.3606  
1   43.6623  -79.3895  
2   43.6572  -79.3789  
3   43.6515  -79.3754  
4   43.6764   -79.293  
5   43.6448  -79.3733  
6    43.658  -79.3874  
7   43.6695  -79.4226  
8   43.6506  -79.3846  
9    43.669  -79.4423  
10  43.6408  -79.3818  
11  43.6479  -79.4197  
12  43.6796  -79.3522  
13  43.6472  -79.3816  
14  43.6368  -79.4282  
15   43.669  -79.3156  
16  43.6482  -79.3798  
17  43.6595  -79.3409  
18   43.728  -79.3888  
19  43.7117  -79.4169  
20  43.7128  -79.3902  
21  43.6969  -79.4113  
22  43.6616  -79.4648  
23  43.7154  -79.4057  
24  43.6727  -79.4057  
25   43.649  -79.4563  
26  43.7043  -79.3888  
27  43.6627     -79.4  
28  43.6516  -79.4844  
29  43.6896  -79.3832  
30  43.6532     -79.4  
31  43.6864     -79.4  
32  43.6289  -79.3944  
33  43.6796  -79.3775  
34  43.6464  -79.3748  
35   43.668  -79.3677  
36  43.6484  -79.3823  
37  43.6659  -79.3832  
38  43.6627  -79.3216

#### Longitude and latitude of Toronto

In [38]:
lat_trnt = 43.651070
lon_trnt = -79.347015

#### Create a map of Tpronto with neighborhoods superimposed on top

In [39]:
# create map of Toronto using latitude and longitude values
map_trnt = folium.Map(location=[lat_trnt, lon_trnt], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_trnt['Latitude'], df_trnt['Longitude'], df_trnt['Borough'], df_trnt['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#2233cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trnt)
    
map_trnt

#### Define Foursquare Credentials and Version

In [66]:
CLIENT_ID = 'TKC5I1TJWMRGIADZYG14FAZWYPD1IGKNZ5EWK4PTQJUSACLS' # your Foursquare ID
CLIENT_SECRET = 'N2LTPZHIYBAGUR1PGDUSLREYTE1OXCL2HZBJLVSZ3BFFJSWE' # your Foursquare Secret
VERSION = '20200625'
LIMIT = 30 #Original Limit
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TKC5I1TJWMRGIADZYG14FAZWYPD1IGKNZ5EWK4PTQJUSACLS
CLIENT_SECRET:N2LTPZHIYBAGUR1PGDUSLREYTE1OXCL2HZBJLVSZ3BFFJSWE


#### Function to repeat acquiring the nearby venues for all neighborhood

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
toronto_venues = getNearbyVenues(names=df_trnt['Neighborhood'],
                                   latitudes=df_trnt['Latitude'],
                                   longitudes=df_trnt['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Check the size of acquired dataframe

In [71]:
toronto_venues.shape

(864, 7)

In [72]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                      Venue  Venue Latitude  Venue Longitude  \
0          Roselle Desserts       43.653447       -79.362017   
1             Tandem Coffee       43.653559       -79.361809   
2    Cooper Koo Family YMCA       43.653249       -79.358008   
3       Body Blitz Spa East       43.654735       -79.359874   
4  Dominion Pub and Kitchen       43.656919       -79.358967   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3                  Spa  
4                  Pub

#### Show how many venues are returned

In [73]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            30   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     30   
Christie                                                               16   
Church and Wellesley                                                   30   
Commerce Court, Victoria Hotel                                         30   
Davisville                                                             30   
Davisville North                                                       10   
Dufferin, Dovercourt Village                                           17   
First Canadian Place, Underground city                                 30   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                               30   
Harbourfront East, Union Station, Toronto Islands                      30   
High Park, The Junction South                                          24   
India Bazaar, The Beaches West                                         21   
Kensington Market, Chinatown, Grange Park                              30   
Lawrence Park                                                           4   
Little Portugal, Trinity                                               30   
Moore Park, Summerhill East                                             1   
North Toronto West, Lawrence Park                                      19   
Parkdale, Roncesvalles                                                 15   
Queen's Park, Ontario Provincial Government                            30   
Regent Park, Harbourfront                                              30   
Richmond, Adelaide, King                                               30   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     30   
St. James Town                                                         30   
St. James Town, Cabbagetown                                            30   
Stn A PO Boxes                                                         30   
Studio District                                                        30   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             4   
The Danforth West, Riverdale                                           30   
Toronto Dominion Centre, Design Exchange                               30   
University of Toronto, Harbord                                         30   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             30   
Brockton, Parkdale Village, Exhibition Place                            23   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      30   
Christie                                                                16   
Church and Wellesley                                                    30   
Commerce Court, Victoria Hotel                                          30   
Davisville                                                              30   
Davisville North                    

In [75]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 195 uniques categories.


### Analyize each Neighborhood

In [78]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Airport  Airport Food Court  Airport Lounge  Airport Service  \
0            0        0                   0               0                0   
1            0        0                   0               0                0   
2            0        0                   0               0                0   
3            0        0                   0               0                0   
4            0        0                   0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   ...  Theater  Theme Restaurant  Toy / Game Store  Trail  Train Station  \
0  ...        0                 0                 0      0              0   
1  ...        0                 0                 0      0              0   
2  ...        0                 0                 0      0              0   
3  ...        0                 0                 0      0              0   
4  ...        0                 0                 0      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                              0                 0                      0   
1                              0                 0                      0   
2                              0                 0                      0   
3                              0                 0                      0   
4                              0                 0                      0   

   Wine Bar  Wine Shop  
0         0          0  
1         0          0  
2         0          0  
3         0          0  
4         0          0  

[5 rows x 195 columns]

In [82]:
#toronto_onehot.shape

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [81]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

In [86]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0        Cocktail Bar  0.07
1         Coffee Shop  0.07
2            Beer Bar  0.07
3  Seafood Restaurant  0.07
4              Museum  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.13
1          Breakfast Spot  0.09
2   Performing Arts Venue  0.09
3             Coffee Shop  0.09
4  Furniture / Home Store  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.12
1  Fast Food Restaurant  0.06
2         Auto Workshop  0.06
3                  Park  0.06
4           Pizza Place  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4               Bar  0.06


----Central Bay